# Пытаюсь сократить количество признаков сохранив точность модели

In [9]:
import sys
from pathlib import Path
import joblib
ROOT = Path.cwd().parent
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

from src.data_loader import (
    load_splits,load_processed_data,save_processed_data
)
import time
import numpy as np
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.multiclass import OneVsRestClassifier



### Обучаю модель на 24 признаках 5 признаков удалены('public_speaking_comfort', 'alone_time_preference','creativity', 'emotional_stability', 'stress_handling').

In [3]:
data = load_splits(scaled=False,encoded_y=True)
y_train = data["y_train"]
y_test = data["y_test"]
X_train = data["X_train"]
X_test = data["X_test"]
X_train_pured = load_processed_data("data/processed/X_train_pured.csv")
X_test_pured = load_processed_data("data/processed/X_test_pured.csv")

In [4]:
# Обучаю LogReg с параметром регуляризации c=0.1
pipe = Pipeline([
    ("scaler",StandardScaler()),
    ("clf",LogisticRegression(max_iter=3000,random_state=42,C=0.1))
])

pipe.fit(X_train_pured,y_train)

y_pred = pipe.predict(X_test_pured)

### Метрики на 24 признаках
**Accuracy:** 0.996
| Class | Precision | Recall | F1-score | Support |
|------:|----------:|-------:|---------:|--------:|
| Ambivert    | 1.00      | 0.99   | 0.99     | 1315    |
| Extrovety     | 1.00      | 1.00   | 1.00     | 1371    |
| Introvert     | 1.00      | 1.00   | 1.00     | 1314    |
| **Macro avg** | 1.00 | 1.00 | 1.00 | 4000 |
| **Weighted avg** | 1.00 | 1.00 | 1.00 | 4000 |

# Модель на 24 признаках как и прежде показывает высокую точность


# Применяю L1 регулиризацию для уменьшения признаков

In [5]:
# Пайплан с L1 регуляризацией.
pipe_l1 = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", OneVsRestClassifier(
        LogisticRegression(
            l1_ratio=1,
            C=0.001,
            solver="saga",
            max_iter=1000,
            random_state=42
        )
    ))
])

pipe_l1.fit(X_train, y_train)

# Смотрю признаки которые занулила модель

coefs = np.vstack([
    est.coef_[0] for est in pipe_l1.named_steps["clf"].estimators_
])

coef_abs = np.abs(coefs).sum(axis=0)
removed_features = X_train.columns[coef_abs == 0]

### Признаки которые исключаем после L1 (17 признаков): 
* 'listening_skill' 
* 'empathy' 
* 'creativity' 
* 'organization'
* 'curiosity'
* 'routine_preference'
* 'friendliness'
* 'emotional_stability'
* 'planning'
* 'spontaneity'
* 'sports_interest'
* 'online_social_usage'
* 'travel_desire'
* 'gadget_usage'
* 'work_style_collaborative'
* 'decision_speed'
* 'stress_handling'

In [10]:
# Новые даанные для обучения и тестирования
X_L1_train = X_train.drop(columns=removed_features)
X_L1_test = X_test.drop(columns=removed_features)
# Сохраняю сплиты 
save_processed_data(X_L1_train, "data/processed/X_train_pured(L1).csv")
save_processed_data(X_L1_test, "data/processed/X_test_pured(L1).csv")

WindowsPath('C:/Users/AlexK/OneDrive/Рабочий стол/Классификаци типов личности/data/processed/X_test_pured(L1).csv')

### Обучаю модель на 12 Признаках

In [7]:
pipe_2 = Pipeline([
    ("scaler",StandardScaler()),
    ("clf",LogisticRegression(max_iter=3000,random_state=42,C=0.1))
])

pipe_2.fit(X_L1_train,y_train)
y_pred_2 = pipe_2.predict(X_L1_test)


In [8]:
# Сохраняю модель 
ARTIFACTS_DIR = Path("../models").resolve()

joblib.dump(pipe_2, ARTIFACTS_DIR / "log_reg_12Features.pkl")

['C:\\Users\\AlexK\\OneDrive\\Рабочий стол\\Классификаци типов личности\\models\\log_reg_12Features.pkl']

### Метрики на 12 признаках

**Accuracy**  0.995
| Class | Precision | Recall | F1-score | Support |
|-------|-----------|--------|----------|---------|
| 0     | 0.99      | 0.99   | 0.99     | 1315    |
| 1     | 1.00      | 0.99   | 1.00     | 1371    |
| 2     | 1.00      | 1.00   | 1.00     | 1314    |
| **Macro avg** | 0.99  | 1.00   | 0.99     | 4000    |
| **Weighted avg** | 1.00 | 0.99 | 1.00     | 4000    |

### Модель на 12 признаках так же демонстрирует высокую точность
### Следуюшим этапом проведу интерпритацию признаков